In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np

# chargement jeu de donnée avec le moins de features
df = pd.read_csv('movies_clean_v2.csv')
df.head(20)
df.shape


(4544, 17)

In [4]:
# df_ml = df.drop(['acteurs', 'realisateur', 'studio'], axis=1)
# df_ml

,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,remake,titre,season,coeff_studio,scoring_acteurs,scoring_acteurs&realisateur
0,315000000,James Horner,2009-12-16,0,Franchise,Science Fiction,Etats-Unis,NaN,NaN,Avatar,Hiver,3,0.000000,0.275862
1,130000000,John Williams,2004-06-02,0,Franchise,Fantasy,Etats-Unis,NaN,NaN,Harry Potter et le prisonnier d'Azkaban,Été,3,0.724138,0.724138
2,200000000,John Williams,2017-12-13,0,Franchise,Fantasy,Etats-Unis,Kathleen Kennedy,NaN,Star Wars: Les derniers Jedi,Hiver,3,0.000000,0.000000
3,295000000,Alan Silvestri,2018-04-25,0,Franchise,Comicbook,Etats-Unis,Kevin Feige,NaN,Avengers: Infinity War,Printemps,3,0.689655,0.689655
4,350000000,NaN,2022-12-14,0,Franchise,Science Fiction,Etats-Unis,NaN,NaN,Avatar : la voie de l'eau,Hiver,3,0.448276,0.724138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,40000000,NaN,2011-04-06,0,NaN,Aventure - Action,Etats-Unis,NaN,Remake,Le Flingueur (2011),Printemps,0,0.034483,0.034483
4537,49515804,NaN,2006-10-04,0,NaN,Drame,Allemagne,NaN,NaN,Le Parfum : histoire d'un meurtrier,Automne,0,0.000000,0.000000
4538,27800000,NaN,2018-02-28,0,NaN,Comédie,France,NaN,NaN,La Ch’tite famille,Hiver,2,0.310345,0.517241
4539,2700000,NaN,2020-07-01,0,NaN,Comédie,France,NaN,NaN,Les Parfums,Été,0,0.000000,0.000000


In [4]:
df['franchise_binary'] = df['franchise'].apply(lambda x: 1 if x == 'Franchise' else 0 if pd.isna(x) else np.nan)
df['remake_binary'] = df['remake'].apply(lambda x: 1 if x == 'Remake' else 0 if pd.isna(x) else np.nan)
df = df.drop(['franchise', 'remake'], axis=1)
df



,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,genre,pays,producteur,realisateur,studio,titre,season,coeff_studio,scoring_acteurs&realisateur,franchise_binary,remake_binary
0,"Johnny Depp, Orlando Bloom, Geoffrey Rush, Kei...",225000000,Hans Zimmer,2006-08-02,150,2708112,Aventure - Action,Etats-Unis,Jerry Brukheimer,Gore Verbinski,Walt Disney Pictures,"Pirates des Caraïbes, le secret du coffre maudit",Été,3,0.310345,1,0
1,"Jean Reno, Christian Clavier",23000000,NaN,1998-02-11,118,2655916,Comédie,France,NaN,JeanMarie Poiré,GBVI,Les Visiteurs II - Les Couloirs du temps,Hiver,2,0.103448,1,0
2,"Daniel Radcliffe, Emma Watson, Ralph Fiennes, ...",150000000,John Williams,2005-11-30,153,3088796,Fantasy,Etats-Unis,NaN,Mike Newell,Warner Bros.,Harry Potter et la coupe de feu,Automne,3,0.758621,1,0
3,"Daniel Craig, Gad Elmaleh, Simon Pegg",135000000,John Williams,2011-10-26,107,3158313,Aventure - Action,Etats-Unis,Steven Spielberg,Steven Spielberg,Sony Pictures,Les Aventures de Tintin : Le secret de la Licorne,Automne,3,0.586207,0,0
4,"Robert Downey Jr, Scarlett Johansson, Chris He...",220000000,Alan Silvestri,2012-04-25,142,2041362,Comicbook,Etats-Unis,Kevin Feige,Joss Whedon,Walt Disney Pictures,Marvel's The Avengers,Printemps,3,0.482759,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,"Matt Dillon, Eva Green",6500000,NaN,2019-11-27,106,51977,Drame,France,NaN,NaN,Pathé,Proxima,Automne,2,0.000000,0,0
4540,Jim Carrey,72000000,NaN,2002-05-08,152,51858,Drame,Etats-Unis,NaN,Frank Darabont,Warner Bros.,The Majestic,Printemps,3,0.068966,0,0
4541,"Colin Farrell, Jennifer Connelly, Russell Crow...",60000000,Hans Zimmer,2014-03-12,129,51385,Romance,Etats-Unis,NaN,NaN,Warner Bros.,Un amour d'hiver,Printemps,3,0.655172,0,0
4542,Leïla Bekhti,3600000,NaN,2024-03-13,101,51632,Drame,France,NaN,NaN,Ad vitam,La Nouvelle femme,Printemps,2,0.068966,0,0
